# AKS Automatic + MCP Weather Server Deployment Walkthrough

This notebook orchestrates deployment of the AKS Automatic infrastructure (defined in [`main.bicep`](main.bicep)) and publishes the MCP Weather Server (located in [`mcp-weather-server/src/index.ts`](mcp-weather-server/src/index.ts)) to the cluster.

## Prerequisites
- Azure CLI authenticated: `az login`
- Docker daemon running, logged into your Azure Container Registry (ACR)
- Bicep CLI installed (comes with Azure CLI 2.50+)
- Kubernetes CLI (`kubectl`) installed
- Notebook kernel with network and shell access (e.g., Azure ML compute, local dev machine)

In [ ]:
import json
import os
import pathlib
import subprocess
import tempfile
import time
from typing import Optional
import requests
from azure.identity import DefaultAzureCredential

# --- Configuration ---
RESOURCE_GROUP = "rg-aks-automatic"
LOCATION = "eastus"
DEPLOYMENT_NAME = "aks-automatic"
TEMPLATE_FILE = "main.bicep"
PARAM_FILE = "main.bicepparam"
ACR_NAME = "<your-registry>"  # e.g. myregistry
AKS_CLUSTER_NAME_OVERRIDE = None  # Optional manual override if you set a different clusterName in parameters
IMAGE_NAME = f"{ACR_NAME}.azurecr.io/mcp-weather-server:latest"
PROJECT_ROOT = pathlib.Path.cwd()

APIM_DEPLOYMENT_NAME = "apim-mcp"
APIM_TEMPLATE_FILE = "apim-mcp.bicep"
APIM_PARAM_FILE = "apim-mcp.bicepparam"
APIM_SERVICE_NAME_OVERRIDE = None  # Optional manual override if apim-mcp.bicepparam differs
MCP_BACKEND_URL = "https://<ingress-or-load-balancer-for-mcp>"

AGENT_DIR = PROJECT_ROOT / "agents"
AGENT_REQUIREMENTS = AGENT_DIR / "requirements.txt"
AGENT_SCRIPT = AGENT_DIR / "create_agent.py"
AGENT_RUN_OUTPUT = AGENT_DIR / "agent_invoke_response.json"

AZURE_AI_PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT", "https://<region>.api.azureml.ms")
AZURE_AI_PROJECT_ID = os.getenv("AZURE_AI_PROJECT_ID", "<project-guid>")
AZURE_AI_RESOURCE_ID = os.getenv("AZURE_AI_RESOURCE_ID", "/subscriptions/<sub-id>/resourceGroups/<rg>/providers/Microsoft.MachineLearningServices/workspaces/<workspace>")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4_1")
APIM_WEATHER_API_URL = os.getenv("APIM_WEATHER_API_URL", "https://<service>.azure-api.net/mcp-weather")
APIM_SUBSCRIPTION_KEY = os.getenv("APIM_SUBSCRIPTION_KEY", "<optional-apim-key>")
AGENT_API_VERSION = "2024-10-01-preview"
AGENT_SCOPE = "https://cognitiveservices.azure.com/.default"

def get_access_token(scope: str = AGENT_SCOPE) -> str:
    credential = DefaultAzureCredential()
    token = credential.get_token(scope)
    return token.token

print(f"Resource group: {RESOURCE_GROUP}")
print(f"AKS deployment name: {DEPLOYMENT_NAME}")
print(f"ACR: {ACR_NAME}")
print(f"Image: {IMAGE_NAME}")
print(f"APIM deployment name: {APIM_DEPLOYMENT_NAME}")
print(f"Backend URL for APIM: {MCP_BACKEND_URL}")
print(f"Agent script path: {AGENT_SCRIPT}")
print(f"Azure AI project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")

In [ ]:
# Step 1: Ensure the resource group exists
subprocess.run(["az", "group", "create",
                "--name", RESOURCE_GROUP,
                "--location", LOCATION,
                "--output", "none"], check=True)
print("Resource group ready.")

In [ ]:
# Step 2: Deploy AKS Automatic using Bicep
deploy = subprocess.run(["az", "deployment", "group", "create",
                           "--name", DEPLOYMENT_NAME,
                           "--resource-group", RESOURCE_GROUP,
                           "--template-file", TEMPLATE_FILE,
                           "--parameters", f"@{PARAM_FILE}",
                           "--output", "json"],
                          check=True, capture_output=True, text=True)
deployment_result = json.loads(deploy.stdout)
deployment_outputs = deployment_result.get("properties", {}).get("outputs", {})
print(json.dumps(deployment_outputs, indent=2))

In [ ]:
# Step 3: Capture key outputs for later steps
cluster_name = AKS_CLUSTER_NAME_OVERRIDE or deployment_outputs.get("clusterName", {}).get("value")
workspace_id = deployment_outputs.get("workspaceId", {}).get("value")
print(f"Cluster name: {cluster_name}")
print(f"Log Analytics workspace: {workspace_id}")
if not cluster_name:
    raise ValueError("clusterName output missing; check your deployment parameters and outputs.")

In [ ]:
# Step 4: Attach ACR permissions to the AKS cluster
subprocess.run(["az", "aks", "update",
                "--resource-group", RESOURCE_GROUP,
                "--name", cluster_name,
                "--attach-acr", ACR_NAME], check=True)
print("AKS cluster now has pull permissions for the ACR.")

In [ ]:
# Step 5: Merge AKS credentials into local kubeconfig
subprocess.run(["az", "aks", "get-credentials",
                "--resource-group", RESOURCE_GROUP,
                "--name", cluster_name,
                "--overwrite-existing"], check=True)
print(f"Kubeconfig updated for {cluster_name}.")

In [ ]:
# Step 6: Build and push MCP Weather Server image
server_dir = PROJECT_ROOT / "mcp-weather-server"
subprocess.run(["npm", "install"], cwd=server_dir, check=True)
subprocess.run(["npm", "run", "build"], cwd=server_dir, check=True)
subprocess.run(["docker", "build", "-t", IMAGE_NAME, "."], cwd=server_dir, check=True)
subprocess.run(["az", "acr", "login", "--name", ACR_NAME], check=True)
subprocess.run(["docker", "push", IMAGE_NAME], check=True)
print("Container image pushed to ACR.")

In [ ]:
# Step 7: Deploy MCP Weather Server workload to AKS
manifest_path = PROJECT_ROOT / "k8s" / "mcp-weather.yaml"
manifest_text = manifest_path.read_text()
patched_manifest = manifest_text.replace("<your-registry>.azurecr.io/mcp-weather-server:latest", IMAGE_NAME)
with tempfile.NamedTemporaryFile('w', suffix='.yaml', delete=False) as temp_manifest:
    temp_manifest.write(patched_manifest)
    rendered_manifest_path = temp_manifest.name
subprocess.run(["kubectl", "apply", "-f", rendered_manifest_path], check=True)
print("MCP Weather Server manifest applied.")

In [ ]:
# Step 8: Verify rollout
subprocess.run(["kubectl", "get", "pods", "-l", "app=mcp-weather-server"], check=True)
subprocess.run(["kubectl", "logs", "deploy/mcp-weather-server"], check=False)
print("Review pod status and logs above.")

## Publish MCP Weather through API Management
Set the MCP service endpoint (`MCP_BACKEND_URL`) above to an address reachable by API Management (for example, an Azure Application Gateway, load balancer IP, or ingress hostname that fronts the MCP Weather service). Then run the following steps to provision API Management and wire it to the backend.

In [ ]:
# Step 9: Deploy API Management for MCP Weather
if "<ingress" in MCP_BACKEND_URL or MCP_BACKEND_URL.endswith("for-mcp>"):
    raise ValueError("Set MCP_BACKEND_URL to the live endpoint that fronts the MCP Weather service before deploying APIM.")

apim_deploy = subprocess.run(["az", "deployment", "group", "create",
                              "--name", APIM_DEPLOYMENT_NAME,
                              "--resource-group", RESOURCE_GROUP,
                              "--template-file", APIM_TEMPLATE_FILE,
                              "--parameters", f"@{APIM_PARAM_FILE}",
                              "--parameters", f"mcpBackendUrl={MCP_BACKEND_URL}",
                              "--output", "json"],
                             check=True, capture_output=True, text=True)

apim_result = json.loads(apim_deploy.stdout)
apim_outputs = apim_result.get("properties", {}).get("outputs", {})
print(json.dumps(apim_outputs, indent=2))

In [ ]:
# Step 10: Summarize API Management endpoints
apim_parameters = apim_result.get("properties", {}).get("parameters", {})
apim_service_name = APIM_SERVICE_NAME_OVERRIDE or apim_parameters.get("serviceName", {}).get("value")
print(f"APIM service name: {apim_service_name}")
print("Gateway URL:", apim_outputs.get("gatewayUrl", {}).get("value"))
print("Developer portal URL:", apim_outputs.get("developerPortalUrl", {}).get("value"))
print("Weather API invoke URL:", apim_outputs.get("apiInvokeUrl", {}).get("value"))

In [ ]:
# Step 11: Store APIM outputs for later steps
if not apim_outputs:
    raise ValueError("APIM outputs unavailable. Run Step 9 to deploy API Management first.")

apim_gateway_url = apim_outputs.get("gatewayUrl", {}).get("value")
apim_developer_portal = apim_outputs.get("developerPortalUrl", {}).get("value")
apim_weather_api_url = apim_outputs.get("apiInvokeUrl", {}).get("value")

if apim_weather_api_url:
    APIM_WEATHER_API_URL = apim_weather_api_url
    os.environ["APIM_WEATHER_API_URL"] = APIM_WEATHER_API_URL
    print(f"Stored APIM weather API URL: {APIM_WEATHER_API_URL}")
else:
    print("APIM weather API URL not found in outputs; ensure apim-mcp.bicep succeeded.")

if apim_gateway_url:
    print(f"APIM gateway URL: {apim_gateway_url}")
if apim_developer_portal:
    print(f"APIM developer portal: {apim_developer_portal}")

## Create Azure AI Foundry Agent
Provide Azure AI Foundry project details in the configuration cell (Cell 3) or export them as environment variables before running the steps below. The agent script will call the Azure AI Projects Agents API to register a GPT-4.1 agent that invokes the MCP Weather API via API Management.

In [ ]:
# Step 12: Install agent dependencies
if not AGENT_REQUIREMENTS.exists():
    raise FileNotFoundError(f"Missing requirements file at {AGENT_REQUIREMENTS}.")

subprocess.run(["python", "-m", "pip", "install", "-r", str(AGENT_REQUIREMENTS)], check=True)

print("Agent dependencies installed.")

In [ ]:
# Step 13: Create GPT-4.1 agent in Azure AI Foundry
placeholders = [AZURE_AI_PROJECT_ENDPOINT, AZURE_AI_PROJECT_ID, AZURE_AI_RESOURCE_ID, AZURE_OPENAI_DEPLOYMENT, APIM_WEATHER_API_URL]
if any(token.startswith("http") and "<" in token for token in placeholders) or any("<" in token for token in placeholders):
    raise ValueError("Replace Azure AI / APIM configuration placeholders in Cell 3 or set real values via environment variables before running this step.")

if not AGENT_SCRIPT.exists():
    raise FileNotFoundError(f"Agent script missing at {AGENT_SCRIPT}.")

agent_env = os.environ.copy()
agent_env.update({
    "AZURE_AI_PROJECT_ENDPOINT": AZURE_AI_PROJECT_ENDPOINT,
    "AZURE_AI_PROJECT_ID": AZURE_AI_PROJECT_ID,
    "AZURE_AI_RESOURCE_ID": AZURE_AI_RESOURCE_ID,
    "AZURE_OPENAI_DEPLOYMENT": AZURE_OPENAI_DEPLOYMENT,
    "APIM_WEATHER_API_URL": APIM_WEATHER_API_URL,
})
if APIM_SUBSCRIPTION_KEY and "<" not in APIM_SUBSCRIPTION_KEY:
    agent_env["APIM_SUBSCRIPTION_KEY"] = APIM_SUBSCRIPTION_KEY

subprocess.run(["python", str(AGENT_SCRIPT)], cwd=AGENT_DIR, check=True, env=agent_env)

print("Agent creation request submitted. Inspect agents/agent_response.json for the agent identifier.")

## Test the Azure AI Foundry Agent
Use the agent identifier returned in `agents/agent_response.json` to send a test weather query.

In [ ]:
# Step 14: Invoke the agent with a sample weather request
agent_response_path = AGENT_DIR / "agent_response.json"
if not agent_response_path.exists():
    raise FileNotFoundError("Run Step 13 first so agent_response.json is available.")

agent_data = json.loads(agent_response_path.read_text())
agent_id = agent_data.get("id") or agent_data.get("name") or agent_data.get("agentId")
if not agent_id:
    raise ValueError("Agent identifier not found in agent_response.json. Inspect the file and update this cell if necessary.")

for value in [AZURE_AI_PROJECT_ENDPOINT, AZURE_AI_PROJECT_ID, AZURE_AI_RESOURCE_ID]:
    if value is None or "<" in value:
        raise ValueError("Provide valid Azure AI configuration values (Cell 3 or environment variables) before invoking the agent.")

token = get_access_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "ai-resource-id": AZURE_AI_RESOURCE_ID,
    "ai-project-id": AZURE_AI_PROJECT_ID,
}

invoke_url = f"{AZURE_AI_PROJECT_ENDPOINT}/openai/agents/{agent_id}:invoke?api-version={AGENT_API_VERSION}"
payload = {
    "input": [
        {
            "role": "user",
            "content": "What is the weather like today in Seattle?"
        }
    ]
}

response = requests.post(invoke_url, headers=headers, data=json.dumps(payload), timeout=60)
response.raise_for_status()
invoke_result = response.json()
AGENT_RUN_OUTPUT.write_text(json.dumps(invoke_result, indent=2))

assistant_reply = None
output = invoke_result.get("output") or invoke_result.get("result")
if isinstance(output, list):
    for item in output:
        blocks = item.get("content") if isinstance(item, dict) else None
        if isinstance(blocks, list):
            for block in blocks:
                if isinstance(block, dict):
                    if block.get("type") == "text":
                        assistant_reply = block.get("text") or block.get("value")
                    elif "text" in block and isinstance(block["text"], dict):
                        assistant_reply = block["text"].get("value")
                if assistant_reply:
                    break
        elif isinstance(blocks, str):
            assistant_reply = blocks
        if assistant_reply:
            break
elif isinstance(output, dict):
    assistant_reply = output.get("message") or output.get("text")

print(f"Agent invocation response saved to {AGENT_RUN_OUTPUT}")
if assistant_reply:
    print("\nAgent reply:\n")
    print(assistant_reply)
else:
    print(json.dumps(invoke_result, indent=2))

## Next steps
- Attach your MCP-compatible Agent to the running pod (sidecar or exec).
- Harden networking, RBAC, and secret management for production environments.
- Configure API Management products, subscriptions, and policies to suit your consumers.
- Extend the notebook with teardown steps (`kubectl delete`, `az group delete`) when you're ready to clean up.